In [99]:
import pandas as pd

## Lectura de datos

In [100]:
cruces = pd.read_csv('cruces.csv', sep=';', encoding='latin1')
display(cruces.head())

,Codigo de vía tratado,Literal completo del vial tratado,Clase de la via tratado,Particula de la via tratado,Nombre de la via tratado,Codigo de via que cruza o enlaza,Literal completo del vial que cruza,Clase de la via que cruza,Particula de la via que cruza,Nombre de la via que cruza,Coordenada X (Guia Urbana) cm (cruce),Coordenada Y (Guia Urbana) cm (cruce),Longitud en S R WGS84 (cruce),Latitud en S R WGS84 (cruce)
0,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,2792,CALLE DE CASTIELLO DE JACA ...,CALLE,DE,CASTIELLO DE JACA,44260746,448479753,3º40'43.38'' W,40º30'36.13'' N
1,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,1837,CALLE DE FEDERICO MOMPOU ...,CALLE,DE,FEDERICO MOMPOU,44260732,448479748,3º40'43.43'' W,40º30'36.13'' N
2,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,87100,CALLE DE MARÍA TUBAU ...,CALLE,DE,MARIA TUBAU,44290433,448494484,3º40'30.85'' W,40º30'40.98'' N
3,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,2793,PASEO DE TIERRA DE MELIDE ...,PASEO,DE,TIERRA DE MELIDE,44320997,448510385,3º40'17.9'' W,40º30'46.21'' N
4,127,CALLE DE ISABEL COLBRAND ...,CALLE,DE,ISABEL COLBRAND,2794,CALLE DE PORTOMARÍN ...,CALLE,DE,PORTOMARIN,44322285,448530840,3º40'17.42'' W,40º30'52.83'' N


In [113]:
direcciones = pd.read_csv('direcciones.csv', sep=';', encoding='latin1')
# display(direcciones.head())
# direcciones['Coordenada X (Guia Urbana) cm'].astype('int64')
# print(direcciones['Coordenada X (Guia Urbana) cm'].dtype)
# direcciones['Coordenada Y (Guia Urbana) cm'].dtype = 'int64'
display(direcciones[direcciones['Coordenada X (Guia Urbana) cm'].str.contains('-')])

C:\Users\Alex MGG\AppData\Local\Temp\ipykernel_19240\1629032222.py:1: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  direcciones = pd.read_csv('direcciones.csv', sep=';', encoding='latin1')


ValueError: Cannot mask with non-boolean array containing NA / NaN values

## Limpieza de datos

In [102]:
# Quitamos los espacios innecesarios de los datasets
for df in [cruces, direcciones]:
    for column in df.columns:
        if df[column].dtype == type(object):
            df[column] = df[column].str.strip()

In [103]:
# Ponemos una coordenada común de las glorietas (media de las coordenadas de los cruces)

# Para empezar nos quedamos con un dataset de las glorietas (solo necesitamos la glorieta como primera via)
cruces_glorietas = cruces[cruces['Clase de la via tratado'] == 'GLORIETA']

# Guardamos el número de cruces de cada glorieta en un diccionario, usando value_counts y diferenciando por el código de la vía
num_cruces = {}
for glorieta in cruces_glorietas['Codigo de vía tratado'].unique():
    num_cruces[glorieta] = cruces_glorietas[cruces_glorietas['Codigo de vía tratado'] == glorieta]['Codigo de vía tratado'].value_counts()[glorieta]

# Guardamos la suma de las coordenadas (diferenciamos entre X e Y) de los cruces de cada glorieta en un diccionario
coords_glorietas = {}
for glorieta in cruces_glorietas['Codigo de vía tratado'].unique():
    coords_glorietas[glorieta] = []
    coords_glorietas[glorieta].append(cruces_glorietas[cruces_glorietas['Codigo de vía tratado'] == glorieta]['Coordenada X (Guia Urbana) cm (cruce)'].sum())
    coords_glorietas[glorieta].append(cruces_glorietas[cruces_glorietas['Codigo de vía tratado'] == glorieta]['Coordenada Y (Guia Urbana) cm (cruce)'].sum())

# Calculamos la media de las coordenadas de los cruces de cada glorieta
for glorieta in coords_glorietas:
    coords_glorietas[glorieta][0] = coords_glorietas[glorieta][0] // num_cruces[glorieta]
    coords_glorietas[glorieta][1] = coords_glorietas[glorieta][1] // num_cruces[glorieta]

# Finalmente, en el dataset de cruces, si hay una glorieta en un cruce, ponemos las coordenadas de la glorieta
for index, row in cruces.iterrows():
    if row['Clase de la via tratado'] == 'GLORIETA':
        cruces.iloc[index, cruces.columns.get_loc('Coordenada X (Guia Urbana) cm (cruce)')] = coords_glorietas[row['Codigo de vía tratado']][0]
        cruces.iloc[index, cruces.columns.get_loc('Coordenada Y (Guia Urbana) cm (cruce)')] = coords_glorietas[row['Codigo de vía tratado']][1]
    elif row['Clase de la via que cruza'] == 'GLORIETA':
        cruces.iloc[index, cruces.columns.get_loc('Coordenada X (Guia Urbana) cm (cruce)')] = coords_glorietas[row['Codigo de via que cruza o enlaza']][0]
        cruces.iloc[index, cruces.columns.get_loc('Coordenada Y (Guia Urbana) cm (cruce)')] = coords_glorietas[row['Codigo de via que cruza o enlaza']][1]


In [106]:
# Para el dataset de direcciones, si hay alguna dirección cuyo numero tiene una letra, unificamos las coordenadas
# de las direcciones con el mismo número y la misma calle
print((direcciones['Codigo de via']).dtype)
direcciones[direcciones['Literal de numeracion'].str.contains('[a-zA-Z]')]

# Primero, en la columna de Literal de numeracion, reemplazamos "KM.", " ", "EN", "SA", "NUM" por ""
# for replacement in ['KM.', ' ', 'EN', 'SA', 'NUM']:
#     direcciones['Literal de numeracion'] = direcciones['Literal de numeracion'].str.replace(replacement, '')

# # Creamos una columna en la que almacenamos la direccion de la forma <codigo de via>-<literal de numeracion (sin letras)>
# direcciones['Direccion'] = direcciones['Codigo de via'].astype(str) + '-' + direcciones['Literal de numeracion'].str.replace('[A-Z]', '')

# # Guardamos el numero de vias de cada direccion en un diccionario
# num_vias = {}
# coords_direcciones = {}
# for direccion in direcciones['Direccion'].unique():
#     count = direcciones[direcciones['Direccion'] == direccion]['Direccion'].value_counts()[direccion]
#     if count > 1:
#         num_vias[direccion] = count
#         coords_direcciones[direccion] = []
#         coords_direcciones[direccion].append(direcciones[direcciones['Direccion'] == direccion]['Coordenada X (Guia Urbana) cm'].sum())
#         coords_direcciones[direccion].append(direcciones[direcciones['Direccion'] == direccion]['Coordenada Y (Guia Urbana) cm'].sum())

# # Guardamos la suma de las coordenadas (diferenciamos entre X e Y) de las vias de cada direccion en un diccionario
# # for direccion in direcciones['Direccion'].unique():

# # Calculamos la media de las coordenadas de las vias de cada direccion
# for direccion in coords_direcciones:
#     coords_direcciones[direccion][0] = coords_direcciones[direccion][0] // num_vias[direccion]
#     coords_direcciones[direccion][1] = coords_direcciones[direccion][1] // num_vias[direccion]

# # Finalmente, en el dataset de direcciones, si hay una direccion con varias vias, ponemos las coordenadas de la direccion
# for index, row in direcciones.iterrows():
#     if num_vias[row['Direccion']] > 1:
#         direcciones.iloc[index, direcciones.columns.get_loc('Coordenada X (Guia Urbana) cm')] = coords_direcciones[row['Direccion']][0]
#         direcciones.iloc[index, direcciones.columns.get_loc('Coordenada Y (Guia Urbana) cm')] = coords_direcciones[row['Direccion']][1]




int64


,Codigo de numero,Codigo de via,Clase de la via,Partï¿½cula de la vï¿½a,Nombre de la vï¿½a,Literal de numeracion,Codigo de distrito,Nombre del distrito,Codigo de barrio,Nombre del barrio,...,Coordenada X (Guia Urbana) cm,Coordenada Y (Guia Urbana) cm,Longitud en S R ETRS89 WGS84,Latitud en S R ETRS89 WGS84,Tipo de la via a la que pertenece el numero,Situacion de la via respecto al terreno,Tipo de denominacion de la via,Parcela catastral del numero,Tipologia del numero,Zona de valor del numero
0,31031089,31001337,AUTOVIA,,A-1,KM.001000EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,...,NaN,NaN,3ï¿½40'23.6'' W,40ï¿½29'21.82'' N,Topï¿½nimo,Nivel,Admon,,Parcela,R21S
1,31031088,31001337,AUTOVIA,,A-1,KM.001000SA,16,HORTALEZA,6,VALDEFUENTES,...,NaN,NaN,3ï¿½40'20.75'' W,40ï¿½29'21.45'' N,Topï¿½nimo,Nivel,Admon,,Parcela,R21N
2,31031091,31001337,AUTOVIA,,A-1,KM.001100EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,...,NaN,NaN,3ï¿½39'57.57'' W,40ï¿½29'48.11'' N,Topï¿½nimo,Nivel,Admon,,Parcela,R21S
3,31031090,31001337,AUTOVIA,,A-1,KM.001100SA,16,HORTALEZA,6,VALDEFUENTES,...,NaN,NaN,3ï¿½39'54.87'' W,40ï¿½29'46.28'' N,Topï¿½nimo,Nivel,Admon,,Parcela,R21R
4,31031093,31001337,AUTOVIA,,A-1,KM.001200EN,8,FUENCARRAL-EL PARDO,6,VALVERDE,...,NaN,NaN,3ï¿½39'33.3'' W,40ï¿½30'13.32'' N,Topï¿½nimo,Nivel,Admon,,Parcela,R21O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210358,20045135,725,PASEO,DEL,ZURRON,NUM000049,21,BARAJAS,4,TIMON,...,NaN,NaN,3ï¿½35'5.92'' W,40ï¿½28'10.77'' N,Vï¿½a,Nivel,Pleno,0403807VK5800C,Frente fachada,R23T
210359,20045138,725,PASEO,DEL,ZURRON,NUM000051,21,BARAJAS,4,TIMON,...,NaN,NaN,3ï¿½35'3.33'' W,40ï¿½28'10.78'' N,Vï¿½a,Nivel,Pleno,0602301VK5800D,Frente fachada,R23T
210360,20045141,725,PASEO,DEL,ZURRON,NUM000053,21,BARAJAS,4,TIMON,...,NaN,NaN,3ï¿½35'2.11'' W,40ï¿½28'11.34'' N,Vï¿½a,Nivel,Pleno,0602301VK5800D,Portal,R23T
210361,20045144,725,PASEO,DEL,ZURRON,NUM000055,21,BARAJAS,4,TIMON,...,NaN,NaN,3ï¿½35'1.04'' W,40ï¿½28'11.18'' N,Vï¿½a,Nivel,Pleno,0602302VK5800D,Frente fachada,R23T


## Cosas q hacer:

- Calles con numeros y letras unificarlas en un numero (hacer media de coordenadas)
- Introducir cruces en dataset de direcciones en su lugar correspondiente
- Navegar por los nodos, guardando informacion de las calles que cruzan, coordenadas y nodos adyacentes.
- Crear un grafo con las calles y sus cruces


## Dudas

- ¿Se puede crear una nueva columna?
- ¿Qué hacemos con los valores erróneos?